In [1]:
import numpy as np
import tensorflow as tf
import keras
import random

import matplotlib.pyplot as plt
import sklearn
import pandas as pd

import itertools

In [142]:
def card_print(cards):
    for i in range(len(cards)):
        if type(cards[i]) != str:
            print("move " + str(i)+": " + cards[i].color+ " "+ str(cards[i].number))
        else:
            print("move " + str(i)+": " + cards[i])
        

##MAKE AN OVERALL FUNCTION THAT CONVERTS A LIST OF UNO CARDS INTO THE REQUIRED MATRIX
##OVERALL FUNCTION THAT CONVERTS LIST OF POTENTIAL MOVES INTO THE REQUIRED LIST OF ACTION
##OVERALL FUNCTION THAT GETS THE STATE OF THE GAME

##REMEMBER POP REMOVES THE TOP CARD AT THE END OF THE LIST AT [-1]

class UnoCard:
    def __init__(self, color, number, deck=None,is_wild=False):
        self.color = color
        self.number = number
        self.is_wild = is_wild
        self.deck= deck

        
class UnoDeck:
    def __init__(self,game):
        self.game = game
        self.players = self.game.player_names###most likely self.game.player_names
        
        hands = {}
        ##initalize uno deck
        self.deck = []
        self.colors = ['red', 'blue', 'green', 'yellow']
        self.numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 'skip', 'reverse', 'draw 2']
        self.card_counts = {}
        
        for color in self.colors:
            #print(color)
            for number in self.numbers:
                #print(str(number) + " " + color)# checking for the numbers of cards
                self.deck.append(UnoCard(color, number, self))
        #print(len(self.deck))
        
        ## for the second run we do not include the numbers of 0
        ##run it twice to do it again
        for color in self.colors:
            for number in self.numbers[1:]:
                self.deck.append(UnoCard(color, number,self))
        
                
        ##now we need to add the wild cards
        ##We will initialize it so color is set to wwild draw four and same for color
        for i in range(4):
            self.deck.append(UnoCard('wild','wild',self, True))
        for i in range(4):
            self.deck.append(UnoCard('wild draw 4','wild draw 4', self, True))
            
        
        np.random.shuffle(self.deck)
        self.hands = {}
        self.hands["in pile"] = self.deck
        self.hands["played"] = []
        for i in range(len(self.players)):
            self.hands[self.players[i]] = []##instantiate each dictionary.
            
        ##we hnow have every player in the game and every card created
        
    def pick_up(self,name_from, name_to):
        removed = self.hands[name_from].pop()
        self.hands[name_to].append(removed)
        
        #this function is the only way cards are picked up
        #hence at the end of each pickup if there is nothing left we reshuffle the hand
        if self.is_pickup_empty():
            self.reshuffle_pickup()
        
            
            
    def move_card(self,name_from, name_to, card):
            ##moves a card from one player to another
        self.hands[name_from].remove(card)
            #add to player
        self.hands[name_to].append(card)
    
    def is_pickup_empty(self):
        res = len(self.hands["in pile"])
        if res == 0:
            return True
        else:
            return False
    def reshuffle_pickup(self):
        while len(self.hands['played']) >1:
            self.move_card('played', "in pile", self.hands['played'][0])
        np.random.shuffle(self.hands['in pile'])
        print("deck shuffled")
    def get_state(self, player_name,pot_moves):
        state =[]
        state.append(self.game.top.color)
        #red
        red_num=0
        green_num=0
        blue_num=0
        yell_num=0
        skip_num=0
        rev_num=0
        pl2_num=0
        pl4_num=0
        wild_num=0
        for i in range(len(self.hands[player_name])):
            if type(self.hands[player_name][i].number)==int:#Normal cards will have integer number
                if self.hands[player_name][i].color =='red':
                    red_num+=1
                elif self.hands[player_name][i].color =='green':
                    green_num+=1
                elif self.hands[player_name][i].color =='blue':
                    blue_num+=1
                elif self.hands[player_name][i].color =='yellow':
                    yell_num+=1
            if type(self.hands[player_name][i].number)==str:
                if self.hands[player_name][i].number =='skip':
                    skip_num+=1
                elif self.hands[player_name][i].number =='reverse':
                    rev_num+=1
                elif self.hands[player_name][i].number =='draw 2':
                    pl2_num+=1
                elif self.hands[player_name][i].number =='wild draw 4':
                    pl4_num+=1
                elif self.hands[player_name][i].number =='wild':
                    wild_num+=1
                
        state.append(red_num)
        state.append(green_num)
        state.append(blue_num)
        state.append(yell_num)
        state.append(skip_num)
        state.append(rev_num)
        state.append(pl2_num)
        state.append(pl4_num)
        state.append(wild_num)
        red_num=0
        green_num=0
        blue_num=0
        yell_num=0
        skip_num=0
        rev_num=0
        pl2_num=0
        ##we know have to check the potential moves 
        ##################################################################
        #maybe we dont use pot moves and instead update based on top card
        ##Gets annoying when pick up is active
        for i in range(len(pot_moves)):
            if type(pot_moves[i]) != str:
                #if the card is an integer
                if type(pot_moves[i].number)==int:#normal cards will have int
                    if pot_moves[i].color =='green':
                        green_num+=1
                    elif pot_moves[i].color =='red':
                        red_num +=1
                    elif pot_moves[i].color =='yellow':
                        yell_num +=1
                    elif pot_moves[i].color =='blue':
                        blue_num+=1
                if type(pot_moves[i].number)==str:
                    if pot_moves[i].number =='skip':
                        skip_num +=1
                    elif pot_moves[i].number =='reverse':
                        rev_num+=1
                    elif pot_moves[i].number =='draw 2':
                        pl2_num +=1
        state.append(red_num)
        state.append(green_num)
        state.append(blue_num)
        state.append(yell_num)
        state.append(skip_num)
        state.append(rev_num)
        state.append(pl2_num)
        #print(state)
        #print(len(state))
        for i in range(1,len(state)):
            if state[i]>2:
                state[i]=2
            if i >=5 and state[i]>1:
                state[i]=1
        print(state)
        return tuple(state)
        ##we now have a funtion that should allow us to attempt to get the state for thecurrent player
        
class Player:
    def __init__(self, name, game):
        
        self.name = name
        self.game = game
        self.deck = self.game.deck
    

    def current_moves(self, top):
        current_moves =[]
            ##THIS FUNCTION ASSUMES THEY CAN PUT DOWN and dont have to pick up 
        current_cards = self.deck.hands[self.name]
        for i in range(len(current_cards)):
            if current_cards[i].color == top.color:
                current_moves.append(current_cards[i])
            elif current_cards[i].number == top.number:
                current_moves.append(current_cards[i])
            elif current_cards[i].is_wild:
                current_moves.append(current_cards[i])
        return current_moves
                
            
            
    def have_plus_2(self):
        ##returns the list of cards that are plus 2
        plus_2 = []
        current_cards = self.deck.hands[self.name]
        for i in range(len(current_cards)):
            if current_cards[i].number == 'draw 2':
                plus_2.append(current_cards[i])
        return plus_2
    
    def have_plus_4(self):
        ##return the list of cards that are plus 4
        plus_4 = []
        current_cards = self.deck.hands[self.name]
        for i in range(len(current_cards)):
            if current_cards[i].number == 'wild draw 4':
                plus_4.append(current_cards[i])
        return plus_4

        
                
            
            
        #def matrix_converter(self):
            ##calls current moves and converts to a matrix, of all cards on x axis, 1 means playable, 0 means not playable
            ##the next column on the x axis will be the probabilities any given card that is not mine someone has.
            ##This will also include the option to pick up most likely
            
        

class UnoGame:
    def __init__(self, num_players, auto_game= False, player_nam = []):
        self.num_players = num_players
        self.game_in_play = True#game isnt finished
        self.player_names = []
        self.is_reversed = False
        self.current_player_state=()# a tuple returning the state of the game before the last player played their hand.
        self.prev_action =""#the action the current player is going to make. from the previous sate
        self.pot_moves = []##this after 
        self.move_cur_player_makes=0#after get action is called on the round this is the chosen index of the move
        self.pickup_active = False#turns to true if a pick up is active 
        self.pickup_num = 0#the number of rounds an active pick up has been active
        if auto_game == False:
            for i in range(num_players):
                player_name = input(f"Enter the name of player {i+1}: ")
                self.player_names.append(player_name)
            self.players = {}
        if auto_game == True:
            #player_names must be input into the function
            self.player_names = player_nam
            self.players = {}
            
        
        self.deck = UnoDeck(self)##initialize an uno game passing itself as a variable
        self.current_player = 0#a number# if 3 players (0,1,2) are potential options
        ##to access current player do self.deck.hands[self.player_names[self.current_player]]
        ##returns the list of the current players current hands
        
        ##instantiate the variables of players
        for i in range(num_players):
            self.players[self.player_names[i]] = Player(self.player_names[i],self)
    
    def deal(self,num_cards = 7):
        for i in range(num_cards):
            for name in self.player_names:
                self.deck.pick_up("in pile", name)
                
        ##here we initialize the first card
        i = 1
        self.top = self.deck.hands['in pile'][-i]#a card objet of the first card
        while self.deck.hands['in pile'][-i].is_wild:
            
            self.top = self.deck.hands['in pile'][-i]
            i+=1#this needs to update each time 
        self.top = self.deck.hands['in pile'][-i]
        self.deck.move_card('in pile', 'played', self.deck.hands['in pile'][-i])
        #need to change the top so it reflects properly the deck
    
    ##Igues we can assume always that there is the option to pick up
    def play_round(self,player_name):
        player_temp = self.players[player_name]
        potential_moves = []
        if self.pickup_active == True:
            if self.top.number == "draw 2":
                potential_moves = player_temp.have_plus_2()
            elif self.top.number == 'wild draw 4':
                potential_moves = player_temp.have_plus_4()
                
        #I think we can all assume these are the only options
        else:
            potential_moves = player_temp.current_moves(self.top)
        return potential_moves
        

    ##change color of wild card when played and pick up is False
    
    #def play_uno(self):
        ##use % to rotate player number if no reverse if reversed
        #game_in_play = True
        #while game_in_play:
        
        ##OPTION 0 WILL ALWAYS BE PICK UP ONE CARD
        ##OPTIONS FROM THERE WILL BE 
        #while True: #for testing purposes
            #current player  name
            #skip_round = False
            #self.play_real_round()
    ###################################################################################
    ## PLAY A SINGLE ROUND ############################################################
    def get_state_pot_moves(self, type_rnd= "randm"):
        #type_rnd= random, player input or Q table logic
        '''
        This function can be called multiple times on a round without updating any parts of the game
        
        '''

        skip_round = False#if
    
        curre_player_name = self.player_names[self.current_player]

        ##we now have the potential moves stored
        print("the current player is:"+str(self.player_names[self.current_player]))
        print("the top card is: " + self.top.color + " "+ str(self.top.number))

        print("the current cards are in the hand are: ")
        card_print(self.deck.hands[self.player_names[self.current_player]])

        #print("The potential moves are: ")
        #card_print(pot_moves)

        ###############

        #APPEND THE OTHER OPTIONS SAY MAYBE PICK UP
        if self.pickup_active:
            if self.top.number =="wild draw 4":
                pot_moves = self.players[curre_player_name].have_plus_4()#should return the player object of current player
                if len(pot_moves) == 0:
                       pot_moves.append("wild draw 4")
            elif self.top.number =="draw 2":
                pot_moves = self.players[curre_player_name].have_plus_2()
                if len(pot_moves) == 0:
                    pot_moves.append("draw 2")
        else:
            ##aplways append the options otherwise to draw one card
            pot_moves = self.play_round(curre_player_name)
            ##I THINK THE ISSUE IS IT CHECKS THE COLOR as well so it thinks you can put a red 4 on a red pllus 2

            pot_moves.append("draw 1")


        ##here we have to check of any of the moves are wild then there will be an option for each color
        for i in range(len(pot_moves)):
            if type(pot_moves[i]) != str:#make sure it is not an str pick up
                if pot_moves[i].color == 'wild draw 4' or pot_moves[i].color== 'wild':
                    #the reason we check color is because this will determine if it is manually added via the insert/
                    #append below, and we only want original cards that are not extras added
                    ## we must remove this
                    card_add = pot_moves[i]

                        ##append 4 cardsd of the different color
                    pot_moves.insert(0, UnoCard(color ='red', number = pot_moves[i].number,is_wild = True))
                    pot_moves.insert(0, UnoCard(color ='green', number = pot_moves[i+1].number,is_wild = True))
                    pot_moves.insert(0, UnoCard(color ='blue', number = pot_moves[i+2].number,is_wild = True))
                    pot_moves.insert(0, UnoCard(color ='yellow', number = pot_moves[i+3].number,is_wild = True))
                    if pot_moves[i+4].color== 'wild':#used later
                        self.colorless_wild=pot_moves[i+4]#this is to store for later when we remove the card from hand to deck
                    elif pot_moves[i+4].color == 'wild draw 4':
                        self.colorless_wild_draw_4 = pot_moves[i+4]#used later
                    pot_moves.remove(pot_moves[i+4])#4 extra options have been added
        #add to player
        print("the final potential moves are: ")
        card_print(pot_moves)

        self.current_player_state=self.deck.get_state(curre_player_name,pot_moves= pot_moves)#sets the current player stateto be accessible
        self.pot_moves = pot_moves
        return self.current_player_state
        
        
        
        
    def select_action(self, type_rnd= "randm",epsi=0.3):
        """
        Input: type rnd = randm, Q_max, player select
        returns an action. number between and 0 that represents the chosen action
        
        """
        curre_player_name = self.player_names[self.current_player]
        #############################################################################################
        #HERE IS WHERE THE SELECTION OF POTENTIAL MOVES HAPPENS
        #############################################################################################
        pot_moves = self.pot_moves
        if type_rnd == "randm":# PLAY RANDOM ROUND
            move_cur_player_makes =random.randint(0,len(pot_moves)-1)
            print("the move chosen is: ")
            card_print([pot_moves[move_cur_player_makes]])
            #make sure current move is stored in move_cur_player_makes
        ##############################################################################################
        elif type_rnd == "Q_max":
            ##play a Q table version
            ##it is the maximum from the row of the state that is a playable move.
            ##create a list of all the possible actions
            """
            red, green, blue, yellow, skip, 
            reverse, draw 2, wild draw 4, wild,pik
            10 different actions
            
        
            """
            #reuse code from the get_state function
            
            action_select = [0,0,0,0,0,0,0,0,0,1]#always will be pick up option
            for i in range(10,len(self.current_player_state)):
                if self.current_player_state[i]>0:
                    action_select[i-10]=1#it is in the same order as the actions themselves
            ##now if they have a wild card in their hands.
            for i in range(len(pot_moves)):
                if type(pot_moves[i]) !=str:
                    if pot_moves[i].number=="wild":
                        action_select[7]=1
                    elif pot_moves[i]=="wild_draw_4":
                        action_select[8]=1
            #we now should have a 1 or a 0 if the actions selected
            # now all we need to do is do a dot product of this too get the thing to check the max value.
            print("state Q =: ")
            print(Q_table.loc[[self.current_player_state]])
            print("the potential actions are: ")
            print(np.array(action_select))
            #print((np.array(action_select)*Q_table.loc[[self.current_player_state]].values[0]).tolist())
            play_q_max = (np.array(action_select)*Q_table.loc[[self.current_player_state]].values[0]).tolist()
            print("Q max: ")
            print(play_q_max)
            max_index = play_q_max.index(max(play_q_max))#storing the index of the max element
            print("max index is: "+str(max_index))
            def action_pot_moves_convert(pot_moves,max_index):
                #converts an action from the 10 digit encoding to the pot_moves
                
                if max_index<=3:
                    #it is a normal card
                    for i in range(len(pot_moves)):
                        if type(pot_moves[i]) != str:
                            if pot_moves[i].color == actions()[max_index] and type(pot_moves[i].number)==int:
                                return i
                elif max_index>3 and max_index<=8:
                    #it is a special card
                    for i in range(len(pot_moves)):
                        if type(pot_moves[i]) != str:
                            if type(pot_moves[i].number) == str and pot_moves[i].number == actions()[max_index]:
                                return i
                elif max_index == 9:
                    #pick up is always added at the end.
                    return len(pot_moves)-1
            print("move index: "+str(action_pot_moves_convert(pot_moves,max_index)))
            move_cur_player_makes = action_pot_moves_convert(pot_moves,max_index)
            
                    
        elif type_rnd == "player_input":
            ##playergives input
            a=False
        ##############################################################################################

        
        action = pot_moves[move_cur_player_makes]
        if type(action)==str:
            self.prev_action="pik"
        else:
            if type(action.number)==int:
                self.prev_action=action.color
            elif type(action.number)==str:
                #it is a spection
                self.prev_action = action.number
        self.move_cur_player_makes = move_cur_player_makes
        return self.prev_action
    
        #########we now have stored the state before the player played and the associated action
        #how the function will update the current player for the next round.
        
    def play_game(self):  
        #############################################################################################
        skip_round = False#assume it isnt a skip of next player first then check later
        ##the move cur player makes will update based on an ai or an algorithm or user input
        pot_moves = self.pot_moves
        move_cur_player_makes = self.move_cur_player_makes
        curre_player_name = self.player_names[self.current_player]
        print("move chosen play game: ")
        card_print([pot_moves[move_cur_player_makes]])
        if type(pot_moves[move_cur_player_makes]) == str:##WILL BE A PICK UP MOVE

            #first check if the pick up number is 1, this is required as otherwise pickup num will be 0
            #as no previous pick up card is played it will be 0 but needs to be 1 for the pickup num to multiply correctly
            if int(pot_moves[move_cur_player_makes][-1]) ==1:
                self.pickup_num =1
            for i in range(self.pickup_num*int(pot_moves[move_cur_player_makes][-1])):#the last value of the string 
            #is the pickup number
            #pick up 1 card each time

            #pick ups are only way to make pickup inactive
                self.deck.pick_up('in pile', curre_player_name)
            #pickups are the only way to make inactive and always will make it inactive for next round.
            self.pickup_active = False
            self.pickup_num = 0
        else:
            #we are now going through the options that will play the card
            card_played = pot_moves[move_cur_player_makes]

            #update the top card
            self.top = card_played#do this before the top card is updated back to be colorless for the card transfer
            if card_played.number == "draw 2":
                self.pickup_active = True
                self.pickup_num +=1
            if card_played.number == "wild draw 4":
                self.pickup_active = True
                self.pickup_num +=1
                card_played = self.colorless_wild_draw_4#this updates the colorless wild so it can be moved within the hands
                #as the hands do not have the color as 4 options would be required
            if card_played.number == "reverse":
                self.is_reversed = not self.is_reversed
            if card_played.number == 'skip':
                skip_round = True
            if card_played.number =="wild":
                card_played = self.colorless_wild


            self.deck.move_card(curre_player_name,'played',card_played)
        ##how the skip round works is it works at the end, not at the start that is why it is default set to false at the
        #top
        #so we check if he played a skip card, then enxt we update the next player then for the next round we update
        #the default back to false




        ##now we need the logic to update the player number
        if skip_round:
            if self.is_reversed:
                self.current_player = (self.current_player-1)%self.num_players##two iterations of reverse update
                self.current_player = (self.current_player-1)%self.num_players
            else:
                self.current_player = (self.current_player+1)%self.num_players
                self.current_player = (self.current_player+1)%self.num_players

        else:
            if self.is_reversed:
                self.current_player = (self.current_player-1)%self.num_players#logic if reversed

            else:  
                self.current_player = (self.current_player+1)%self.num_players#logic if not reversed and not plus 2

        #check if current player has won based on the curre player name
        
        
        #if len(self.deck.hands[curre_player_name])==0:
            #player jas won
            #game_in_play = False
            #print("game finished")
            #print("player " + str(curre_player_name)+"has won! YAAYYYY UR SO GOOD")
            #break

In [144]:
Game1 = UnoGame(2,auto_game =True,player_nam = ["Rory","Isaac"])
Game1.deal()
Game1.get_state_pot_moves()#get state seems to be working well
Game1.select_action(type_rnd= "Q_max",epsi=0.3)
#Game1.play_game()
#Game1.get_state_pot_moves()
##we are instead going to have three functions. one that gets the state, then one that gets the action,
##if it is either user input, random or Q_table training. I guess later there will be one that will be Q_table only
##the Qtable one will be annoying because we will need to set epsilon and stuff
##The last function will take, action and make that action and play out the game logic.
##this way we get current state, generate next state and have action all seperately
##The first function will return the potential moves and state

the current player is:Rory
the top card is: blue 4
the current cards are in the hand are: 
move 0: wild wild
move 1: yellow 3
move 2: blue 7
move 3: red 4
move 4: green 3
move 5: wild wild
move 6: red 7
the final potential moves are: 
move 0: yellow wild
move 1: blue wild
move 2: green wild
move 3: red wild
move 4: blue 7
move 5: red 4
move 6: wild wild
move 7: draw 1
['blue', 2, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]
state Q =: 
                                                         red     green  \
(blue, 2, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0...  0.453513  0.190731   

                                                       blue    yellow  \
(blue, 2, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0...  0.06144  0.858775   

                                                        skip  reverse  \
(blue, 2, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0...  0.509729  0.32412   

                                                      draw 2  wild draw 4  \
(blue, 2, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 

'red'

In [125]:
state  = ['blue', 1, 0, 1, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0]
for i in range(1,len(state)):
    if state[i]>2:
        state[i]=2
    if i >=5 and state[i]>1:
        state[i]=1
state

['blue', 1, 0, 1, 2, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [85]:
def actions():
    """TODO: Comment this function"""

    actions_all = [
        "red", "green", "blue", "yellow", "skip", 
        "reverse", "draw 2", "wild draw 4", "wild","pik"
    ]    
    return actions_all
def rewards(states, actions):
    """TODO: Comment this function"""
    
    R = np.zeros((len(states), len(actions)))
    states_t = [min(sum(states[i][1:10]),1) for i in range(len(states))]

    for i in range(len(states)):
        if states_t[i] == 0:
            R[i] = 1

    R = pd.DataFrame(
        data=R, 
        columns=actions, 
        index=states)

    return R
def states():
    """TODO: Comment this function"""

    # Normal cards
    norm_cards = {"red":2,"green":2,"blue":2,"yellow":2}
    spec_cards = {"skip":1,"reverse":1,"draw 2":1}
    wild_cards = {"Wild draw 4":1,"wild":1}

    # Special cards
    norm_cards_play = {"red#":1,"green#":1,"blue#":1,"yellow#":1}
    spec_cards_play = {"skip#":1,"reverse#":1,"draw 2#":1}
    ##what we notice is after the 4th index

    # Combine dictionaries
    states_dict  = {
        **norm_cards, 
        **spec_cards, 
        **wild_cards, 
        **norm_cards_play, 
        **spec_cards_play
    }
    states = [["red", "green", "blue", "yellow"]]

    for val in states_dict.values():
        aux = range(0,val+1)
        states.append(aux)

    # Conduct all combinations
    states = list(itertools.product(*states))
    states_all = list()

    for i in range(len(states)):
        if (
            states[i][1] >= states[i][10] and
            states[i][2] >= states[i][11] and
            states[i][3] >= states[i][12] and
            states[i][4] >= states[i][13] and
            states[i][5] >= states[i][14] and
            states[i][6] >= states[i][15] and
            states[i][7] >= states[i][16]
        ): 
            states_all.append(states[i])

    return states_all

def rewards(states, actions):
    """TODO: Comment this function"""
    
    R = np.zeros((len(states), len(actions)))
    states_t = [min(sum(states[i][1:10]),1) for i in range(len(states))]

    for i in range(len(states)):
        if states_t[i] == 0:
            R[i] = 1

    R = pd.DataFrame(
        data=R, 
        columns=actions, 
        index=states)

    return R
def Q_tables(states, actions):
    """TODO: Comment this function"""
    
    R = np.random.rand(len(states), len(actions))
    states_t = [min(sum(states[i][1:10]),1) for i in range(len(states))] 

    R = pd.DataFrame(
        data=R, 
        columns=actions, 
        index=states)
    return R
def Ns(states, actions):
    """TODO: Comment this function"""
    
    R = np.zeros((len(states), len(actions)))
    states_t = [min(sum(states[i][1:10]),1) for i in range(len(states))] 

    R = pd.DataFrame(
        data=R, 
        columns=actions, 
        index=states)

    return R
state= states()
action = actions()
reward = rewards(state,action)
Q_table = Q_tables(state,action)
N = Ns(state,action)

In [86]:
Q_table

,red,green,blue,yellow,skip,reverse,draw 2,wild draw 4,wild,pik
"(red, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",0.784092,0.496861,0.404237,0.629490,0.878865,0.711815,0.218762,0.666453,0.011495,0.244415
"(red, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0)",0.721148,0.738870,0.112333,0.567586,0.787217,0.934970,0.815959,0.252552,0.924772,0.410302
"(red, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0)",0.396744,0.626574,0.039249,0.073304,0.035097,0.191681,0.556141,0.007049,0.193594,0.811766
"(red, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0)",0.548243,0.351750,0.485384,0.709359,0.219637,0.091298,0.881892,0.490261,0.590267,0.482670
"(red, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0)",0.505041,0.308572,0.105878,0.482230,0.838340,0.840584,0.373298,0.595371,0.949177,0.842106
...,...,...,...,...,...,...,...,...,...,...
"(yellow, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1)",0.555615,0.093160,0.761482,0.197514,0.512824,0.160937,0.128301,0.810792,0.419493,0.906376
"(yellow, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0)",0.024323,0.496343,0.065392,0.850386,0.954237,0.647571,0.832879,0.117640,0.035150,0.784344
"(yellow, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1)",0.893786,0.562656,0.761900,0.055585,0.270143,0.995090,0.809883,0.273278,0.997939,0.494999
"(yellow, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0)",0.654044,0.997021,0.867652,0.888470,0.314328,0.250799,0.671137,0.382222,0.586815,0.443200


In [121]:
state = ("blue", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
Q_table.loc[[state]]

,red,green,blue,yellow,skip,reverse,draw 2,wild draw 4,wild,pik
"(blue, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",0.964551,0.701546,0.690429,0.908226,0.183903,0.238588,0.828911,0.043251,0.723083,0.196673


In [110]:
action_select = [0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
(np.array(action_select)*Q_table.loc[[Game1.current_player_state]].values[0]).tolist()
##glitch in the code, if there is multiple plus 4s that are playable


KeyError: "None of [Index([('blue', 1, 0, 1, 2, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0)], dtype='object')] are in the [index]"

In [120]:
Game1.current_player_state

('blue', 1, 0, 1, 2, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0)

In [119]:
len(("red", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))

17